In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import linalg
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

# Data
class dataset:
    
    def __init__(self, filename):
        self.filename = filename
        self.data = self.loadData()
        
    # load data
    # Input: file path, data type, delimiter
    # Output: a numpy array
    def loadData(self, dt = str, deli =','):
        data = np.loadtxt(self.filename, dtype = dt, delimiter = deli)
        # Replace the missing data '?' with '0'
        for line in data:
            line[line == '?'] = '0'
        return data.astype(int)
    
    # Split the data into train and test randomly with specified ratio 
    # Input: data, ratio
    # Output: four numpy array: X_train, X_test, y_train, y_test
    def split(self, ratio = 0.2):
        n = self.data.shape[0]
        np.random.shuffle(self.data)
        # the size of data Multiple split ratio
        offset = int(n * ratio)
        # X is a list of features (remove sample ID), y is a list of labels
        X_train, X_test, y_train, y_test = self.data[offset:,1:-1], self.data[:offset,1:-1], \
        self.data[offset:, -1], self.data[:offset,-1]
        # Using 0, 1 Instead of 2, 4 of labels
        y_train = [0 if (x == 2) else 1 for x in y_train]
        y_test = [0 if (x == 2) else 1 for x in y_test]
        return X_train, X_test, y_train, y_test

    
# Model   
class mineLogisticRegression:
    def __init__(self, X, y):
        # n is the number of features
        _, n = X.shape
        # Initial weight values [w0, w1, ... , wn] with 0
        self.estimatedParas = np.zeros(n+1)
        self.alpha = 0.001/(len(y))
    
    # Calculate estimation values p(x) using possibility function (logistical regression model)
    def calProbability(self, X):
        e = np.exp(np.dot(self.estimatedParas, X.T))
        return e/(1 + e)
    
    # Calculate first derivation of log likelihood
    def firstDerivative(self, X, y, py):
        # L2 regularization
        regular1 = self.alpha * self.estimatedParas
        # sum(y_i - p(x) * x_ij)
        return np.dot((y - py),X) + regular1
     
    # Calculate second derivation of log likelihood
    def secondDerivative(self, X, py):
        # L2 regularization
        regular2 = self.alpha * np.eye(len(self.estimatedParas))
        # sum(x_ij * x_jk * p(x_i) * (1 - p(x_i)))
        prob = py*(1-py)
        xProb = np.array([i*j for (i,j) in zip(X, prob)])
        return -1*np.dot(xProb.T, X) + regular2
 
    # Update estimated parameters by using Newton Method
    def newtonMethodParaUpdate(self, derivative1, derivative2):
        # next parameter = last parameter value - derivative2/derivative1
        ratio = np.dot(linalg.inv(derivative2),derivative1)
        self.estimatedParas = self.estimatedParas - ratio
 
    # Fit model 
    # Input: trianing data, max iterations and min increment changes of parameter
    # It will stop training when parameters won't change or 
    # it reaches max iterations or min increment (stable) 
    def fit(self, X, y, maxIteration, minIncrement):
        
        # Store last estimated parameter values
        lastEstimatedParas = []
        
        # calculating intercepts
        intercepts = np.ones((X.shape[0], 1))
        X = np.c_[X, intercepts]
        
        # Initial iteration value
        iteration = 0
 
        # training model
        while(list(self.estimatedParas) != list(lastEstimatedParas)):
            # Record last estimated parameter values
            lastEstimatedParas = self.estimatedParas
            # Estimated y
            py = self.calProbability(X)
            # First derivative
            derivative1 = self.firstDerivative(X, y, py)
            # Second derivative
            derivative2 = self.secondDerivative(X, py)
            # Update estimated parameter value
            self.newtonMethodParaUpdate(derivative1, derivative2)
            # Update iteration value
            iteration += 1
            # Calculate increment value between last estimated parameter and current one
            increment = linalg.norm(self.estimatedParas - lastEstimatedParas)
            # Stop when it reaches max iterations
            if(iteration > maxIteration):
                print("The max iteration has been reached!")
                break
            # Stop when it reaches min increment
            else:
                if(increment <= minIncrement):
                    print("The min increment change of parameters hass been reached!")
                    break
                    
    # Predict the classification result with test dataset
    def predict(self, X):
        # calculating intercepts
        intercepts = np.ones((X.shape[0], 1))
        X = np.c_[X, intercepts]
        # calculating probability
        py = self.calProbability(X)
        # deciding class by probability
        return [0 if x <=0.5 else 1 for x in py]
  
    
# File path
filename = "./breast-cancer-wisconsin.data";
# Create a dataset object, which will load this file
data = dataset(filename)

# Initial average accuracy value over 10 trials for mine implement and Scikit Learn
avg_accuracy1 = 0
avg_accuracy2 = 0

# 10 trials
for i in range(10):
    # Get randomly X_train, X_test, y_train, y_test by split function
    X_train, X_test, y_train, y_test = data.split()
    
    # The logistic Regression model from mine implement
    model1 = mineLogisticRegression(X_train, y_train) 
    # Trainning model
    model1.fit(X_train, y_train, 100, 10**(-4))
    # Predict over test data
    predict1 = model1.predict(X_test)
    # Get accuracy for each trial
    confMatrix1 = metrics.confusion_matrix(y_test, predict1)
    accuracy1  = metrics.accuracy_score(y_test, predict1)
    print(accuracy1)
    # Calculate average accuracy
    avg_accuracy1 += accuracy1/10
    
    # The logistic Regression model from Scikit Learn
    model2 = LogisticRegression(solver = 'newton-cg')
    # Trainning model
    model2.fit(X_train, y_train)
    # Predict over test data
    predict2 = model2.predict(X_test)
    # Get accuracy for each trial
    confMatrix2 = metrics.confusion_matrix(y_test, predict2)
    accuracy2  = metrics.accuracy_score(y_test, predict2)
    print(accuracy2)
    # Calculate average accuracy
    avg_accuracy2 += accuracy2/10

# Print result
print("Mine implement: the average accuracy over 10 trials is ", avg_accuracy1)
print("Scikit Learn: the average accuracy over 10 trials is ", avg_accuracy2)



The min increment change of parameters hass been reached!
0.9640287769784173
0.9640287769784173
The min increment change of parameters hass been reached!
0.9568345323741008
0.9568345323741008
The min increment change of parameters hass been reached!
0.9496402877697842
0.9496402877697842
The min increment change of parameters hass been reached!
0.9928057553956835
0.9928057553956835
The min increment change of parameters hass been reached!
0.9712230215827338
0.9712230215827338
The min increment change of parameters hass been reached!
0.9568345323741008
0.9568345323741008
The min increment change of parameters hass been reached!
0.9856115107913669
0.9856115107913669
The min increment change of parameters hass been reached!
0.9712230215827338
0.9712230215827338
The min increment change of parameters hass been reached!
0.9712230215827338
0.9712230215827338
The min increment change of parameters hass been reached!
0.9784172661870504
0.9784172661870504
Mine implement: the average accuracy ove